In [16]:
import numpy as np  
import sys,os  
caffe_root = '/data/ssd-caffe/py2_caffe/'
sys.path.insert(0, caffe_root + 'python')  
import caffe  

train_proto = 'yolov3-tiny-ncs-without-last-maxpool.prototxt'
#train_model = 'snapshot/mobilenet_iter_7000.caffemodel'  #should be your snapshot caffemodel

'''if(len(sys.argv) < 2):
    print("please input caffemodel file from commandLine")
    exit(1)
train_model = sys.argv[1]'''

deploy_proto = 'yolov3-tiny-ncs-without-last-maxpool-merg-batchnorm.prototxt'  
#save_model = 'iter_7000_MobileNetSSD_deploy.caffemodel'
#save_model = train_model.split('/')[-1] + "MobileNetSSD_deploy.caffemodel"
save_model = "generated_merge.caffemodel"
#global buf #----why can not work ?

def merge_bn(net, nob):
    '''
    merge the batchnorm, scale layer weights to the conv layer, to  improve the performance
    var = var + scaleFacotr
    rstd = 1. / sqrt(var + eps)
    w = w * rstd * scale
    b = (b - mean) * rstd * scale + shift
    '''
    buf = [0, 2, 0, 960000, 0]
    for key in net.params.iterkeys():
        print("key:" + str(key))
        if type(net.params[key]) is caffe._caffe.BlobVec:
            print("is caffe._caffe.BlobVec")
            if key.endswith("-bn") or key.endswith("-scale"):
                continue
            else:
                conv = net.params[key]
                print("merge layer {0}".format(conv))
                if not net.params.has_key(key[:-5] + "-bn"):
                    print("not need bn, copy w and b")
                    for i, w in enumerate(conv):
                        nob.params[key][i].data[...] = w.data
                        print(w.data.shape)
                        temp_array = w.data.reshape(-1)                       
                        buf = buf + temp_array.tolist()
                        print(len(buf))
                else:
                    
                    print("need bn, really start bn and scale")
                    bn = net.params[key[:-5] + "-bn"]
                    scale = net.params[key[:-5] + "-scale"]
                    wt = conv[0].data
                    channels = wt.shape[0]
                    bias = np.zeros(wt.shape[0])
                    if len(conv) > 1:
                        bias = conv[1].data
                    mean = bn[0].data
                    var = bn[1].data
                    scalef = bn[2].data
                    print('scalef = {0}'.format(scalef))

                    scales = scale[0].data
                    shift = scale[1].data

                    if scalef != 0:
                        scalef = 1. / scalef
                    mean = mean * scalef
                    var = var * scalef
                    rstd = 1. / np.sqrt(var + 1e-5)
                    rstd1 = rstd.reshape((channels,1,1,1))
                    scales1 = scales.reshape((channels,1,1,1))
                    wt = wt * rstd1 * scales1
                    bias = (bias - mean) * rstd * scales + shift
                    
                    nob.params[key][0].data[...] = wt
                    print(wt.shape)
                    temp_array = wt.reshape(-1)
                    buf = buf + temp_array.tolist()
                    nob.params[key][1].data[...] = bias
                    print(bias.shape)
                    temp_array = bias.reshape(-1)
                    buf = buf + temp_array.tolist()
                    print(len(buf))
    weights_merged_bn = np.array(buf, dtype=np.float32)
    print(weights_merged_bn)
    fp = open('aaa.weights', "wb")
    weights_merged_bn.tofile(fp)
    #np.save('npsaved_file', weights_merged_bn)

#net = caffe.Net(train_proto, train_model, caffe.TRAIN)
caffe.set_mode_cpu()
net = caffe.Net(train_proto, 'Jenerated_nolastpooling.caffemodel', caffe.TEST)
net_deploy = caffe.Net(deploy_proto, caffe.TEST)  

merge_bn(net, net_deploy)
net_deploy.save(save_model)


key:layer1-conv
is caffe._caffe.BlobVec
merge layer <caffe._caffe.BlobVec object at 0x7ff91e96f830>
need bn, really start bn and scale
scalef = [ 1.]
(16, 3, 3, 3)
(16,)
453
key:layer1-bn
is caffe._caffe.BlobVec
key:layer1-scale
is caffe._caffe.BlobVec
key:layer3-conv
is caffe._caffe.BlobVec
merge layer <caffe._caffe.BlobVec object at 0x7ff929db4e50>
need bn, really start bn and scale
scalef = [ 1.]
(32, 16, 3, 3)
(32,)
5093
key:layer3-bn
is caffe._caffe.BlobVec
key:layer3-scale
is caffe._caffe.BlobVec
key:layer5-conv
is caffe._caffe.BlobVec
merge layer <caffe._caffe.BlobVec object at 0x7ff929db4c20>
need bn, really start bn and scale
scalef = [ 1.]
(64, 32, 3, 3)
(64,)
23589
key:layer5-bn
is caffe._caffe.BlobVec
key:layer5-scale
is caffe._caffe.BlobVec
key:layer7-conv
is caffe._caffe.BlobVec
merge layer <caffe._caffe.BlobVec object at 0x7ff929db4590>
need bn, really start bn and scale
scalef = [ 1.]
(128, 64, 3, 3)
(128,)
97445
key:layer7-bn
is caffe._caffe.BlobVec
key:layer7-scale
is

In [10]:
help(np.ndarray.tofile)

Help on method_descriptor:

tofile(...)
    a.tofile(fid, sep="", format="%s")
    
    Write array to a file as text or binary (default).
    
    Data is always written in 'C' order, independent of the order of `a`.
    The data produced by this method can be recovered using the function
    fromfile().
    
    Parameters
    ----------
    fid : file or str
        An open file object, or a string containing a filename.
    sep : str
        Separator between array items for text output.
        If "" (empty), a binary file is written, equivalent to
        ``file.write(a.tobytes())``.
    format : str
        Format string for text file output.
        Each entry in the array is formatted to text by first converting
        it to the closest Python type, and then using "format" % item.
    
    Notes
    -----
    This is a convenience function for quick storage of array data.
    Information on endianness and precision is lost, so this method is not a
    good choice for files in